In [1]:
import numpy as np
import glob
import h5py

from keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [0]:
dataset_path = '/content/drive/My Drive/MachineLearning/Synthesize/dataset'
output_path = '/content/drive/My Drive/MachineLearning/Synthesize/output'

In [0]:
import numpy as np

import h5py
import time

import matplotlib.pylab as plt
import matplotlib.pyplot as plot

import keras.backend as K
from keras.utils import generic_utils
from keras.optimizers import Adam, SGD

from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation, Lambda, Reshape
from keras.layers.convolutional import Conv2D, Deconv2D, ZeroPadding2D, UpSampling2D
from keras.layers import Input, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
import keras.backend as K
from keras.models import load_model

%matplotlib inline

In [0]:
datasetpath = output_path + '/dataset.hdf5'
patch_size = 32
batch_size = 256
epoch = 200
loss_list = []

In [0]:
def normalization(X):
    return X / 127.5 - 1

def load_data(datasetpath):
    with h5py.File(datasetpath, "r") as hf:
        X_full_train = hf["TrainWithoutTarget"][:].astype(np.float32)
        X_full_train = normalization(X_full_train)
        X_sketch_train = hf["TrainWithTarget"][:].astype(np.float32)
        X_sketch_train = normalization(X_sketch_train)
        X_full_val = hf["TestWithoutTarget"][:].astype(np.float32)
        X_full_val = normalization(X_full_val)
        X_sketch_val = hf["TestWithTarget"][:].astype(np.float32)
        X_sketch_val = normalization(X_sketch_val)
        return X_full_train, X_sketch_train, X_full_val, X_sketch_val

In [0]:
def conv_block_unet(x, f, name, bn_axis, bn=True, strides=(2,2)):
    x = LeakyReLU(0.2)(x)
    x = Conv2D(f, (3,3), strides=strides, name=name, padding='same')(x)
    if bn: x = BatchNormalization(axis=bn_axis)(x)
    return x

def up_conv_block_unet(x, x2, f, name, bn_axis, bn=True, dropout=False):
    x = Activation('relu')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(f, (3,3), name=name, padding='same')(x)
    if bn: x = BatchNormalization(axis=bn_axis)(x)
    if dropout: x = Dropout(0.5)(x)
    x = Concatenate(axis=bn_axis)([x, x2])
    return x

In [0]:
def generator_unet_upsampling(img_shape, disc_img_shape, model_name="generator_unet_upsampling"):
    filters_num = 64
    axis_num = -1
    channels_num = img_shape[-1]
    min_s = min(img_shape[:-1])

    unet_input = Input(shape=img_shape, name="unet_input")

    conv_num = int(np.floor(np.log(min_s)/np.log(2)))
    list_filters_num = [filters_num*min(8, (2**i)) for i in range(conv_num)]

    # Encoder
    first_conv = Conv2D(list_filters_num[0], (3,3), strides=(2,2), name='unet_conv2D_1', padding='same')(unet_input)
    list_encoder = [first_conv]
    for i, f in enumerate(list_filters_num[1:]):
        name = 'unet_conv2D_' + str(i+2)
        conv = conv_block_unet(list_encoder[-1], f, name, axis_num)
        list_encoder.append(conv)

    # prepare decoder filters
    list_filters_num = list_filters_num[:-2][::-1]
    if len(list_filters_num) < conv_num-1:
        list_filters_num.append(filters_num)

    # Decoder
    first_up_conv = up_conv_block_unet(list_encoder[-1], list_encoder[-2],
                        list_filters_num[0], "unet_upconv2D_1", axis_num, dropout=True)
    list_decoder = [first_up_conv]
    for i, f in enumerate(list_filters_num[1:]):
        name = "unet_upconv2D_" + str(i+2)
        if i<2:
            d = True
        else:
            d = False
        up_conv = up_conv_block_unet(list_decoder[-1], list_encoder[-(i+3)], f, name, axis_num, dropout=d)
        list_decoder.append(up_conv)

    x = Activation('relu')(list_decoder[-1])
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(disc_img_shape[-1], (3,3), name="last_conv", padding='same')(x)
    x = Activation('tanh')(x)

    generator_unet = Model(inputs=[unet_input], outputs=[x])
    return generator_unet

In [0]:
def DCGAN_discriminator(img_shape, disc_img_shape, patch_num, model_name='DCGAN_discriminator'):
    disc_raw_img_shape = (disc_img_shape[0], disc_img_shape[1], img_shape[-1])
    list_input = [Input(shape=disc_img_shape, name='disc_input_'+str(i)) for i in range(patch_num)]
    list_raw_input = [Input(shape=disc_raw_img_shape, name='disc_raw_input_'+str(i)) for i in range(patch_num)]

    axis_num = -1
    filters_num = 64
    conv_num = int(np.floor(np.log(disc_img_shape[1])/np.log(2)))
    list_filters = [filters_num*min(8, (2**i)) for i in range(conv_num)]

    # First Conv
    generated_patch_input = Input(shape=disc_img_shape, name='discriminator_input')
    xg = Conv2D(list_filters[0], (3,3), strides=(2,2), name='disc_conv2d_1', padding='same')(generated_patch_input)
    xg = BatchNormalization(axis=axis_num)(xg)
    xg = LeakyReLU(0.2)(xg)

    # First Raw Conv
    raw_patch_input = Input(shape=disc_raw_img_shape, name='discriminator_raw_input')
    xr = Conv2D(list_filters[0], (3,3), strides=(2,2), name='raw_disc_conv2d_1', padding='same')(raw_patch_input)
    xr = BatchNormalization(axis=axis_num)(xr)
    xr = LeakyReLU(0.2)(xr)

    # Next Conv
    for i, f in enumerate(list_filters[1:]):
        name = 'disc_conv2d_' + str(i+2)
        x = Concatenate(axis=axis_num)([xg, xr])
        x = Conv2D(f, (3,3), strides=(2,2), name=name, padding='same')(x)
        x = BatchNormalization(axis=axis_num)(x)
        x = LeakyReLU(0.2)(x)

    x_flat = Flatten()(x)
    x = Dense(2, activation='softmax', name='disc_dense')(x_flat)

    PatchGAN = Model(inputs=[generated_patch_input, raw_patch_input], outputs=[x], name='PatchGAN')

    x = [PatchGAN([list_input[i], list_raw_input[i]]) for i in range(patch_num)]

    if len(x)>1:
        x = Concatenate(axis=axis_num)(x)
    else:
        x = x[0]

    x_out = Dense(2, activation='softmax', name='disc_output')(x)

    discriminator_model = Model(inputs=(list_input+list_raw_input), outputs=[x_out], name=model_name)

    return discriminator_model

In [0]:
def DCGAN(generator, discriminator, img_shape, patch_size):
    raw_input = Input(shape=img_shape, name='DCGAN_input')
    genarated_image = generator(raw_input)

    h, w = img_shape[:-1]
    ph, pw = patch_size, patch_size

    list_row_idx = [(i*ph, (i+1)*ph) for i in range(h//ph)]
    list_col_idx = [(i*pw, (i+1)*pw) for i in range(w//pw)]

    list_gen_patch = []
    list_raw_patch = []
    for row_idx in list_row_idx:
        for col_idx in list_col_idx:
            raw_patch = Lambda(lambda z: z[:, row_idx[0]:row_idx[1], col_idx[0]:col_idx[1], :])(raw_input)
            list_raw_patch.append(raw_patch)
            x_patch = Lambda(lambda z: z[:, row_idx[0]:row_idx[1], col_idx[0]:col_idx[1], :])(genarated_image)
            list_gen_patch.append(x_patch)

    DCGAN_output = discriminator(list_gen_patch+list_raw_patch)

    DCGAN = Model(inputs=[raw_input],
                  outputs=[genarated_image, DCGAN_output],
                  name='DCGAN')

    return DCGAN

In [0]:
def load_generator(img_shape, disc_img_shape):
    model = generator_unet_upsampling(img_shape, disc_img_shape)
    return model

def load_DCGAN_discriminator(img_shape, disc_img_shape, patch_num):
    model = DCGAN_discriminator(img_shape, disc_img_shape, patch_num)
    return model

def load_DCGAN(generator, discriminator, img_shape, patch_size):
    model = DCGAN(generator, discriminator, img_shape, patch_size)
    return model

In [0]:
def l1_loss(y_true, y_pred):
    return K.sum(K.abs(y_pred - y_true), axis=-1)

def inverse_normalization(X):
    return (X + 1.) / 2.

def to3d(X):
    if X.shape[-1]==3: return X
    b = X.transpose(3,1,2,0)
    c = np.array([b[0],b[0],b[0]])
    return c.transpose(3,1,2,0)

In [0]:
def plot_generated_batch(X_proc, X_raw, generator_model, batch_size, suffix):
    X_gen = generator_model.predict(X_raw)
    X_raw = inverse_normalization(X_raw)
    X_proc = inverse_normalization(X_proc)
    X_gen = inverse_normalization(X_gen)

    Xs = to3d(X_raw[:5])
    Xg = to3d(X_gen[:5])
    Xr = to3d(X_proc[:5])
    Xs = np.concatenate(Xs, axis=1)
    Xg = np.concatenate(Xg, axis=1)
    Xr = np.concatenate(Xr, axis=1)
    XX = np.concatenate((Xs,Xg,Xr), axis=0)

    plt.imshow(XX)
    plt.axis('off')
    plt.savefig(output_path + '/current_batch_'+suffix+'.png')
    plt.clf()
    plt.close()

In [0]:
def extract_patches(X, patch_size):
    list_X = []
    list_row_idx = [(i*patch_size, (i+1)*patch_size) for i in range(X.shape[1] // patch_size)]
    list_col_idx = [(i*patch_size, (i+1)*patch_size) for i in range(X.shape[2] // patch_size)]
    for row_idx in list_row_idx:
        for col_idx in list_col_idx:
            list_X.append(X[:, row_idx[0]:row_idx[1], col_idx[0]:col_idx[1], :])
    return list_X

def get_disc_batch(procImage, rawImage, generator_model, batch_counter, patch_size):
    if batch_counter % 2 == 0:
        # produce an output
        X_disc = generator_model.predict(rawImage)
        y_disc = np.zeros((X_disc.shape[0], 2), dtype=np.uint8)
        y_disc[:, 0] = 1
    else:
        X_disc = procImage
        y_disc = np.zeros((X_disc.shape[0], 2), dtype=np.uint8)

    X_disc = extract_patches(X_disc, patch_size)
    return X_disc, y_disc

In [0]:
def train():
    # load data
    rawImage, procImage, rawImage_val, procImage_val = load_data(datasetpath)

    img_shape = rawImage.shape[-3:]
    patch_num = (img_shape[0] // patch_size) * (img_shape[1] // patch_size)
    disc_img_shape = (patch_size, patch_size, procImage.shape[-1])

    # train
    opt_dcgan = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt_discriminator = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # load generator model
    generator_model = load_generator(img_shape, disc_img_shape)
    # load discriminator model
    discriminator_model = load_DCGAN_discriminator(img_shape, disc_img_shape, patch_num)

    generator_model.compile(loss='mae', optimizer=opt_discriminator)
    discriminator_model.trainable = False

    DCGAN_model = load_DCGAN(generator_model, discriminator_model, img_shape, patch_size)

    loss = [l1_loss, 'binary_crossentropy']
    loss_weights = [1E1, 1]
    DCGAN_model.compile(loss=loss, loss_weights=loss_weights, optimizer=opt_dcgan)

    discriminator_model.trainable = True
    discriminator_model.compile(loss='binary_crossentropy', optimizer=opt_discriminator)

    # start training
    print('start training')
    for e in range(epoch):

        starttime = time.time()
        perm = np.random.permutation(rawImage.shape[0])
        X_procImage = procImage[perm]
        X_rawImage  = rawImage[perm]
        X_procImageIter = [X_procImage[i:i+batch_size] for i in range(0, rawImage.shape[0], batch_size)]
        X_rawImageIter  = [X_rawImage[i:i+batch_size] for i in range(0, rawImage.shape[0], batch_size)]
        b_it = 0
        progbar = generic_utils.Progbar(len(X_procImageIter)*batch_size)
        for (X_proc_batch, X_raw_batch) in zip(X_procImageIter, X_rawImageIter):
            b_it += 1
            X_disc, y_disc = get_disc_batch(X_proc_batch, X_raw_batch, generator_model, b_it, patch_size)
            raw_disc, _ = get_disc_batch(X_raw_batch, X_raw_batch, generator_model, 1, patch_size)
            x_disc = X_disc + raw_disc
            # update the discriminator
            disc_loss = discriminator_model.train_on_batch(x_disc, y_disc)

            # create a batch to feed the generator model
            idx = np.random.choice(procImage.shape[0], batch_size)
            X_gen_target, X_gen = procImage[idx], rawImage[idx]
            y_gen = np.zeros((X_gen.shape[0], 2), dtype=np.uint8)
            y_gen[:, 1] = 1

            # Freeze the discriminator
            discriminator_model.trainable = False
            gen_loss = DCGAN_model.train_on_batch(X_gen, [X_gen_target, y_gen])
            # Unfreeze the discriminator
            discriminator_model.trainable = True

            progbar.add(batch_size, values=[
                ("D logloss", disc_loss),
                ("G tot", gen_loss[0]),
                ("G L1", gen_loss[1]),
                ("G logloss", gen_loss[2])
            ])

            if b_it % (procImage.shape[0]//batch_size) == 0:
                loss_list.append(gen_loss[0])
                
            # save images for visualization
            if b_it % (procImage.shape[0]//batch_size//2) == 0 and e % 10 == 0:
                plot_generated_batch(X_proc_batch, X_raw_batch, generator_model, batch_size, "training"+str(e))
                idx = np.random.choice(procImage_val.shape[0], batch_size)
                X_gen_target, X_gen = procImage_val[idx], rawImage_val[idx]
                plot_generated_batch(X_gen_target, X_gen, generator_model, batch_size, "validation"+str(e))
                generator_model.save(output_path + '/gen_model.h5')

        print("")
        print('Epoch %s/%s, Time: %s' % (e + 1, epoch, time.time() - starttime))
        if (e+1) % 2 == 0:          
            height = np.array(range(e+1))
            plot.plot(height, loss_list)
            plot.xlabel("Epoch")
            plot.ylabel("Loss")
            plot.savefig(output_path + '/loss.png')
    
    height = np.array(range(epoch))
    plot.plot(height, loss_list)
    plot.xlabel("Epoch")
    plot.ylabel("Loss")
    plot.savefig(output_path + '/loss.png')
    plot.show()
        
    generator_model.save(output_path + '/gen_model.h5')
    
    from keras.models import load_model

#     gen_model = load_model(output_path + '/gen_model.h5')
    img = load_img('1.jpg', target_size=(64,64))
    array = img_to_array(img)
    
    in_array = []
    in_array.append(array)
    in_array = np.array(in_array)    
    
    test = generator_model.predict(in_array)
    X_gen = inverse_normalization(test)
    Xg = to3d(X_gen)
    Xg = np.concatenate(Xg, axis=1)

    plt.imshow(Xg)
    plt.axis('off')
    plt.savefig(output_path + '/predict.png')
    plt.clf()
    plt.close()

In [0]:
loss_list = []
train()

OSError: ignored

In [0]:
def resume_train():
    # load data
    rawImage, procImage, rawImage_val, procImage_val = load_data(datasetpath)

    img_shape = rawImage.shape[-3:]
    patch_num = (img_shape[0] // patch_size) * (img_shape[1] // patch_size)
    disc_img_shape = (patch_size, patch_size, procImage.shape[-1])

    # train
    opt_dcgan = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt_discriminator = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # load generator model
#     generator_model = load_generator(img_shape, disc_img_shape)
    generator_model = load_model(output_path + '/gen_model.h5')
    # load discriminator model
    discriminator_model = load_DCGAN_discriminator(img_shape, disc_img_shape, patch_num)

#     generator_model.compile(loss='mae', optimizer=opt_discriminator)
    discriminator_model.trainable = False

    DCGAN_model = load_DCGAN(generator_model, discriminator_model, img_shape, patch_size)

    loss = [l1_loss, 'binary_crossentropy']
    loss_weights = [1E1, 1]
    DCGAN_model.compile(loss=loss, loss_weights=loss_weights, optimizer=opt_dcgan)

    discriminator_model.trainable = True
    discriminator_model.compile(loss='binary_crossentropy', optimizer=opt_discriminator)

    # start training
    print('start training')
    for e in range(epoch):

        starttime = time.time()
        perm = np.random.permutation(rawImage.shape[0])
        X_procImage = procImage[perm]
        X_rawImage  = rawImage[perm]
        X_procImageIter = [X_procImage[i:i+batch_size] for i in range(0, rawImage.shape[0], batch_size)]
        X_rawImageIter  = [X_rawImage[i:i+batch_size] for i in range(0, rawImage.shape[0], batch_size)]
        b_it = 0
        progbar = generic_utils.Progbar(len(X_procImageIter)*batch_size)
        for (X_proc_batch, X_raw_batch) in zip(X_procImageIter, X_rawImageIter):
            b_it += 1
            X_disc, y_disc = get_disc_batch(X_proc_batch, X_raw_batch, generator_model, b_it, patch_size)
            raw_disc, _ = get_disc_batch(X_raw_batch, X_raw_batch, generator_model, 1, patch_size)
            x_disc = X_disc + raw_disc
            # update the discriminator
            disc_loss = discriminator_model.train_on_batch(x_disc, y_disc)

            # create a batch to feed the generator model
            idx = np.random.choice(procImage.shape[0], batch_size)
            X_gen_target, X_gen = procImage[idx], rawImage[idx]
            y_gen = np.zeros((X_gen.shape[0], 2), dtype=np.uint8)
            y_gen[:, 1] = 1

            # Freeze the discriminator
            discriminator_model.trainable = False
            gen_loss = DCGAN_model.train_on_batch(X_gen, [X_gen_target, y_gen])
            # Unfreeze the discriminator
            discriminator_model.trainable = True

            progbar.add(batch_size, values=[
                ("D logloss", disc_loss),
                ("G tot", gen_loss[0]),
                ("G L1", gen_loss[1]),
                ("G logloss", gen_loss[2])
            ])

            if b_it % (procImage.shape[0]//batch_size) == 0:
                loss_list.append(gen_loss[0])
                
            # save images for visualization
            if b_it % (procImage.shape[0]//batch_size) == 0 and e % 10 == 0:
                plot_generated_batch(X_proc_batch, X_raw_batch, generator_model, batch_size, "training"+str(e))
                idx = np.random.choice(procImage_val.shape[0], batch_size)
                X_gen_target, X_gen = procImage_val[idx], rawImage_val[idx]
                plot_generated_batch(X_gen_target, X_gen, generator_model, batch_size, "validation"+str(e))
                generator_model.save(output_path + '/gen_model.h5')

        print("")
        print('Epoch %s/%s, Time: %s' % (e + 1, epoch, time.time() - starttime))
        if e % 100 == 0:          
            height = np.array(range(len(loss_list)))
            plot.plot(height, loss_list)
            plot.xlabel("Epoch")
            plot.ylabel("Loss")
            plot.savefig(output_path + '/loss.png')
    
    height = np.array(range(epoch))
    plot.plot(height, loss_list)
    plot.xlabel("Epoch")
    plot.ylabel("Loss")
    plot.savefig(output_path + '/loss.png')
    plot.show()
        
    generator_model.save(output_path + '/gen_model.h5')

In [0]:
loss_list = []
resume_train()

start training
30208/30208 [==============================] - 171s 6ms/step - D logloss: 0.6043 - G tot: 1.4019 - G L1: 0.0193 - G logloss: 1.2088

Epoch 1/200, Time: 172.95369625091553
30208/30208 [==============================] - 143s 5ms/step - D logloss: 0.5821 - G tot: 1.5297 - G L1: 0.0203 - G logloss: 1.3263

Epoch 2/200, Time: 145.3279149532318
30208/30208 [==============================] - 143s 5ms/step - D logloss: 0.5765 - G tot: 1.5697 - G L1: 0.0242 - G logloss: 1.3279

Epoch 3/200, Time: 144.03214836120605
30208/30208 [==============================] - 143s 5ms/step - D logloss: 0.5404 - G tot: 1.4518 - G L1: 0.0401 - G logloss: 1.0506

Epoch 4/200, Time: 143.98797082901
30208/30208 [==============================] - 143s 5ms/step - D logloss: 0.5266 - G tot: 1.2672 - G L1: 0.0338 - G logloss: 0.9295

Epoch 5/200, Time: 143.8042435646057
30208/30208 [==============================] - 143s 5ms/step - D logloss: 0.5585 - G tot: 1.3878 - G L1: 0.0492 - G logloss: 0.8958

Ep

In [0]:
from PIL import Image

img = Image.open('current_batch_validation.png')
img_array = np.asarray(img)
plt.axis('off')
plt.imshow(img_array)
plt.show()

#学習済みモデルで予測

In [0]:
from google.colab import files
uploaded = files.upload()

Saving 0.jpg to 0.jpg


In [0]:
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array, array_to_img, save_img
from keras.models import load_model

test_path = '/content/drive/My Drive/MachineLearning/Synthesize/Test/'
gen_model = load_model(test_path + 'gen_model.h5')

OSError: ignored

In [0]:
img = load_img(test_path + '0.jpg', target_size=(64,64))
array = img_to_array(img)
print(array)

In [0]:
in_array = []
in_array.append(array)
in_array = np.array(in_array)
print(in_array)

In [0]:
start = time.time()
norm = normalization(in_array)
test = gen_model.predict(norm)
test = inverse_normalization(test)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
# print(test)
Xg = to3d(test[:1])
Xg = np.concatenate(Xg, axis=1)
pred = array_to_img(Xg)
save_img(test_path + 'predict.jpg', pred)

# plt.imshow(Xg)
# plt.axis('off')
# plt.savefig(test_path + 'predict.jpg')
# plt.clf()
# plt.close()

elapsed_time:0.017178058624267578[sec]


In [20]:
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array, array_to_img, save_img
from keras.models import load_model

test_path = '/content/drive/My Drive/Research/test/real/'
pred_path = test_path + 'predict/'
data_path = test_path + 'crop/'
gen_model = load_model( '/content/drive/My Drive/Research/test/real/gen_model.h5')

for i in range(9):
    #load
    img = load_img(data_path + str(i) + '.jpg', target_size=(128,128))
    array = img_to_array(img)
    in_array = []
    in_array.append(array)
    in_array = np.array(in_array)

    #predict
    start = time.time()
    norm = normalization(in_array)
    test = gen_model.predict(norm)
    test = inverse_normalization(test)
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time) + "[sec]")

    #save
    Xg = to3d(test[:1])
    Xg = np.concatenate(Xg, axis=1)
    pred = array_to_img(Xg)
    save_img(pred_path + str(i) + '.jpg', pred)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


elapsed_time:9.228650093078613[sec]
elapsed_time:0.02676987648010254[sec]
elapsed_time:0.02577376365661621[sec]
elapsed_time:0.027028322219848633[sec]
elapsed_time:0.02607560157775879[sec]
elapsed_time:0.026736974716186523[sec]
elapsed_time:0.0263822078704834[sec]
elapsed_time:0.025943994522094727[sec]
elapsed_time:0.02462911605834961[sec]
